# Import + Load

In [1]:
import pandas as pd
import re
import json
import numpy as np
import ast
import time

In [2]:
df = pd.read_csv("movies/movies_metadata.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

/tmp/ipykernel_26359/3629544706.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("movies/movies_metadata.csv")


In [3]:
df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


# Assessments

Let's try to find columns with multiple values on each of its rows.

In [4]:
# Function to identify columns with multiple values
def identify_multi_value_columns(df):
    multi_value_columns = []
    pattern = re.compile(r'{')  # Pattern to match multi-value entries

    for column in df.columns:
        if df[column].astype(str).apply(lambda x: bool(pattern.search(x))).any():
            multi_value_columns.append(column)
    
    return multi_value_columns

multi_value_columns = identify_multi_value_columns(df)
print("Columns with multiple values:", multi_value_columns)

Columns with multiple values: ['belongs_to_collection', 'genres', 'homepage', 'original_title', 'overview', 'production_companies', 'production_countries', 'spoken_languages']


It's odd that we have `belongs_to_collection`, `homepage`, `original_title`, and `overview` in the list. The others are plural so they made sense. Let's analyze them one-by-one.

## 1. Weird Original Titles

Some original_title values are JSON, and they don't have titles.

In [5]:
df[df['original_title'].str.contains(r'{', regex=True)][['original_title', 'title']]

,original_title,title
19730,"[{'iso_639_1': 'en', 'name': 'English'}]",NaN
29503,"[{'iso_639_1': 'ja', 'name': '日本語'}]",NaN
35587,"[{'iso_639_1': 'en', 'name': 'English'}]",NaN


One of the rows looks like this.

In [6]:
df.loc[19730]

adult                                                   - Written by Ørnås
belongs_to_collection                                             0.065736
budget                                    /ff9qCepilowshEtG2GYWwzt2bs4.jpg
genres                   [{'name': 'Carousel Productions', 'id': 11176}...
homepage                 [{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...
id                                                              1997-08-20
imdb_id                                                                  0
original_language                                                    104.0
original_title                    [{'iso_639_1': 'en', 'name': 'English'}]
overview                                                          Released
popularity                                                             NaN
poster_path                                                   Midnight Man
production_companies                                                 False
production_countries     

`value_counts()` of `original_title` to get an idea of how it should look like.

In [7]:
df['original_title'].value_counts(dropna=False)

original_title
Hamlet                         8
Alice in Wonderland            8
A Christmas Carol              7
Cinderella                     7
Les Misérables                 7
                              ..
Dracula: Dead and Loving It    1
The American President         1
GoldenEye                      1
Sudden Death                   1
Tom and Huck                   1
Name: count, Length: 43373, dtype: int64

How about other titles. Are there any NaNs?

In [8]:
df[df['title'].isna()][['title', 'original_title', 'poster_path']]

,title,original_title,poster_path
19729,NaN,Midnight Man,NaN
19730,NaN,"[{'iso_639_1': 'en', 'name': 'English'}]",Midnight Man
29502,NaN,マルドゥック・スクランブル 排気,NaN
29503,NaN,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Mardock Scramble: The Third Exhaust
35586,NaN,Avalanche Sharks,NaN
35587,NaN,"[{'iso_639_1': 'en', 'name': 'English'}]",Avalanche Sharks


Is there any NaN `original_title`?

In [9]:
df[df['original_title'].isna()]['original_title']

Series([], Name: original_title, dtype: object)

There is no NaN `original_title`.

### Conclusion

For all NaN `title`s:

- Use the `original_title` if `original_title` does not have JSON text
- Use `poster_path` otherwise, and also replace the `original_title` with `poster_path`

Issue name: **NaN `title` and invalid `original_title`**

## 2. `belongs_to_collection` and `homepage` - Found a different issue instead: There are three invalid rows

### 2.1. `belongs_to_collection`
Let's first check out its value counts

In [10]:
df['belongs_to_collection'].value_counts(dropna=False)

belongs_to_collection
NaN                                                                                                                                                               40972
{'id': 415931, 'name': 'The Bowery Boys', 'poster_path': '/q6sA4bzMT9cK7EEmXYwt7PNrL5h.jpg', 'backdrop_path': '/foe3kuiJmg5AklhtD3skWbaTMf2.jpg'}                    29
{'id': 421566, 'name': 'Totò Collection', 'poster_path': '/4ayJsjC3djGwU9eCWUokdBWvdLC.jpg', 'backdrop_path': '/jaUuprubvAxXLAY5hUfrNjxccUh.jpg'}                    27
{'id': 96887, 'name': 'Zatôichi: The Blind Swordsman', 'poster_path': '/8Q31DAtmFJjhFTwQGXghBUCgWK2.jpg', 'backdrop_path': '/bY8gLImMR5Pr9PaG3ZpobfaAQ8N.jpg'}       26
{'id': 645, 'name': 'James Bond Collection', 'poster_path': '/HORpg5CSkmeQlAolx3bKMrKgfi.jpg', 'backdrop_path': '/6VcVl48kNKvdXOZfJPdarlUGOsk.jpg'}                  26
                                                                                                                                          

Most of them are NaNs with seemingly the rest of them JSON values. Let's find out.

Get all rows where `belongs_to_collection` has JSON-like strings:

In [11]:
df[df['belongs_to_collection'].fillna('').str.contains(r'{', regex=True)][['belongs_to_collection', 'title']]

,belongs_to_collection,title
0,"{'id': 10194, 'name': 'Toy Story Collection', ...",Toy Story
2,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",Grumpier Old Men
4,"{'id': 96871, 'name': 'Father of the Bride Col...",Father of the Bride Part II
9,"{'id': 645, 'name': 'James Bond Collection', '...",GoldenEye
12,"{'id': 117693, 'name': 'Balto Collection', 'po...",Balto
...,...,...
45355,"{'id': 37261, 'name': 'The Carry On Collection...",Carry On Follow That Camel
45358,"{'id': 37261, 'name': 'The Carry On Collection...",Carry On Camping
45369,"{'id': 37261, 'name': 'The Carry On Collection...",Carry On England
45371,"{'id': 477208, 'name': 'DC Super Hero Girls Co...",LEGO DC Super Hero Girls: Brain Drain


And now, get all rows where `belongs_to_collection` is not JSON and not NaN either:

In [12]:
filled = df['belongs_to_collection'].fillna('')
df[(filled.fillna('') != '') & (~filled.str.contains(r'{', regex=True))] \
  [['belongs_to_collection', 'title', 'original_title', 'poster_path']]

,belongs_to_collection,title,original_title,poster_path
19730,0.065736,NaN,"[{'iso_639_1': 'en', 'name': 'English'}]",Midnight Man
29503,1.931659,NaN,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Mardock Scramble: The Third Exhaust
35587,2.185485,NaN,"[{'iso_639_1': 'en', 'name': 'English'}]",Avalanche Sharks


That's interesting. Apparently, there are `belongs_to_collection` values that are floating numbers, and they all have titles in their `poster_path` field. Let's see if there's another field that contains the correct `belongs_to_collection` value.

In [13]:
df.loc[19730]

adult                                                   - Written by Ørnås
belongs_to_collection                                             0.065736
budget                                    /ff9qCepilowshEtG2GYWwzt2bs4.jpg
genres                   [{'name': 'Carousel Productions', 'id': 11176}...
homepage                 [{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...
id                                                              1997-08-20
imdb_id                                                                  0
original_language                                                    104.0
original_title                    [{'iso_639_1': 'en', 'name': 'English'}]
overview                                                          Released
popularity                                                             NaN
poster_path                                                   Midnight Man
production_companies                                                 False
production_countries     

Looks like there isn't any.

Now, finally, let's get a typical value for the JSON-like `belongs_to_collection`.

In [14]:
df.loc[45355]['belongs_to_collection']

"{'id': 37261, 'name': 'The Carry On Collection', 'poster_path': '/2P0HNrYgKDvirV8RCdT1rBSJdbJ.jpg', 'backdrop_path': '/38tF1LJN7ULeZAuAfP7beaPMfcl.jpg'}"

Looks like we need to get the "name" property.

### Conclusion

- For rows with `belongs_to_collection` containing JSON values, replace `belongs_to_collection` with its "name" property.
- For rows with `belongs_to_collection` containing floating numbers, replace them with NaN.

### 2.2. `homepage`

Same drill as before. Do value_counts, and then find json values.

In [15]:
df['homepage'].value_counts(dropna=False)

homepage
NaN                                                  37684
http://www.georgecarlin.com                             12
http://www.wernerherzog.com/films-by.html                7
http://breakblade.jp/                                    6
http://www.crownintlpictures.com/tztitles.html           4
                                                     ...  
http://www.arthurchristmas.com/                          1
http://happyfeettwo.warnerbros.com/index.html            1
http://rampartmovie.com/                                 1
http://singyoursongthemovie.com/                         1
http://www.cinemadpresents.com/2011/04/shit-year/        1
Name: count, Length: 7674, dtype: int64

In [16]:
df[df['homepage'].fillna('').str.contains(r'{', regex=True)][['homepage', 'title', 'poster_path']]

,homepage,title,poster_path
4635,http://tartanvideo.com/film.asp?ProjectID={C66...,L.I.E.,/ck5Tkq17M6khEN7JQlQznoYl37L.jpg
19730,"[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",NaN,Midnight Man
29503,"[{'iso_3166_1': 'US', 'name': 'United States o...",NaN,Mardock Scramble: The Third Exhaust
35587,"[{'iso_3166_1': 'CA', 'name': 'Canada'}]",NaN,Avalanche Sharks


![three](three.jpg)

Looks like the values in these three rows are incorrect. Let's see them in details.

In [17]:
display(df.loc[19730]['homepage'])
df.loc[19730]

"[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso_3166_1': 'LU', 'name': 'Luxembourg'}, {'iso_3166_1': 'GB', 'name': 'United Kingdom'}, {'iso_3166_1': 'US', 'name': 'United States of America'}]"

adult                                                   - Written by Ørnås
belongs_to_collection                                             0.065736
budget                                    /ff9qCepilowshEtG2GYWwzt2bs4.jpg
genres                   [{'name': 'Carousel Productions', 'id': 11176}...
homepage                 [{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...
id                                                              1997-08-20
imdb_id                                                                  0
original_language                                                    104.0
original_title                    [{'iso_639_1': 'en', 'name': 'English'}]
overview                                                          Released
popularity                                                             NaN
poster_path                                                   Midnight Man
production_companies                                                 False
production_countries     

In [18]:
df.loc[29503]

adult                     Rune Balot goes to a casino connected to the ...
belongs_to_collection                                             1.931659
budget                                    /zV8bHuSL6WXoD6FWogP9j4x80bL.jpg
genres                   [{'name': 'Aniplex', 'id': 2883}, {'name': 'Go...
homepage                 [{'iso_3166_1': 'US', 'name': 'United States o...
id                                                              2012-09-29
imdb_id                                                                  0
original_language                                                     68.0
original_title                        [{'iso_639_1': 'ja', 'name': '日本語'}]
overview                                                          Released
popularity                                                             NaN
poster_path                            Mardock Scramble: The Third Exhaust
production_companies                                                 False
production_countries     

In [19]:
df.loc[35587]

adult                     Avalanche Sharks tells the story of a bikini ...
belongs_to_collection                                             2.185485
budget                                    /zaSf5OG7V8X8gqFvly88zDdRm46.jpg
genres                   [{'name': 'Odyssey Media', 'id': 17161}, {'nam...
homepage                          [{'iso_3166_1': 'CA', 'name': 'Canada'}]
id                                                              2014-01-01
imdb_id                                                                  0
original_language                                                     82.0
original_title                    [{'iso_639_1': 'en', 'name': 'English'}]
overview                                                          Released
popularity                                           Beware Of Frost Bites
poster_path                                               Avalanche Sharks
production_companies                                                 False
production_countries     

Whereas typical rows should look like this.

In [20]:
df.iloc[1]

adult                                                                False
belongs_to_collection                                                  NaN
budget                                                            65000000
genres                   [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...
homepage                                                               NaN
id                                                                    8844
imdb_id                                                          tt0113497
original_language                                                       en
original_title                                                     Jumanji
overview                 When siblings Judy and Peter discover an encha...
popularity                                                       17.015539
poster_path                               /vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg
production_companies     [{'name': 'TriStar Pictures', 'id': 559}, {'na...
production_countries     

### Conclusion

There are three rows that contained invalid values, and they have many NULL values. We can take a guess on what the correct columns should be, but due to the NULL values and the fact they are only three rows, we can just remove them.

Issue name: **Incorrect rows with ID 19730, 29503, and 35587**

## 3. `overview`

In [21]:
df['overview'].value_counts(dropna=False)

overview
NaN                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    954
No overview found.                                                                                                                  

In [22]:
df[df['overview'].fillna('').str.contains(r'{', regex=True)][['overview', 'title', 'poster_path']]

,overview,title,poster_path
19588,"""Volumen"" collects every video from Bjork's ca...",Björk: Volumen,/uEfwqT0vIA31SPyfEihNRmBpQ1C.jpg
39350,"In Africa, many years ago, Slim and Tom don't ...",I'm for the Hippopotamus,/5fMQdIGO5tncCa7sH26xXOSzAFa.jpg


### Conclusion

For `overview`, we can just keep the values as they are. This is not an issue.

## 4. Other columns with multiple values (`genres`, `production_companies`, `production_countries`, `spoken_languages`)

In [23]:
df['genres']

0        [{'id': 16, 'name': 'Animation'}, {'id': 35, '...
1        [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...
2        [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...
3        [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...
4                           [{'id': 35, 'name': 'Comedy'}]
                               ...                        
45461    [{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...
45462                        [{'id': 18, 'name': 'Drama'}]
45463    [{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...
45464                                                   []
45465                                                   []
Name: genres, Length: 45466, dtype: object

In [24]:
df['production_companies'].value_counts(dropna=False)

production_companies
[]                                                                                                                                                                                      11875
[{'name': 'Metro-Goldwyn-Mayer (MGM)', 'id': 8411}]                                                                                                                                       742
[{'name': 'Warner Bros.', 'id': 6194}]                                                                                                                                                    540
[{'name': 'Paramount Pictures', 'id': 4}]                                                                                                                                                 505
[{'name': 'Twentieth Century Fox Film Corporation', 'id': 306}]                                                                                                                           439
                             

Notice from the `df.loc[19730]` result above that we also have rows with `production_companies=='False'`.

In [25]:
df[df['production_companies'] == 'False']

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
19730,- Written by Ørnås,0.065736,/ff9qCepilowshEtG2GYWwzt2bs4.jpg,"[{'name': 'Carousel Productions', 'id': 11176}...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",1997-08-20,0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29503,Rune Balot goes to a casino connected to the ...,1.931659,/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg,"[{'name': 'Aniplex', 'id': 2883}, {'name': 'Go...","[{'iso_3166_1': 'US', 'name': 'United States o...",2012-09-29,0,68.0,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Released,...,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35587,Avalanche Sharks tells the story of a bikini ...,2.185485,/zaSf5OG7V8X8gqFvly88zDdRm46.jpg,"[{'name': 'Odyssey Media', 'id': 17161}, {'nam...","[{'iso_3166_1': 'CA', 'name': 'Canada'}]",2014-01-01,0,82.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,...,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


The "False" strings need to be replaced by NaN. **However, these rows are going to be deleted anyway, so this is not an issue.**

In [26]:
df['production_countries'].value_counts(dropna=False)

production_countries
[{'iso_3166_1': 'US', 'name': 'United States of America'}]                                                                                                                                     17851
[]                                                                                                                                                                                              6282
[{'iso_3166_1': 'GB', 'name': 'United Kingdom'}]                                                                                                                                                2238
[{'iso_3166_1': 'FR', 'name': 'France'}]                                                                                                                                                        1654
[{'iso_3166_1': 'JP', 'name': 'Japan'}]                                                                                                                                                        

Use both the "iso_3166_1" and "name" properties to replace `production_countries`.

In [27]:
df['spoken_languages'].value_counts(dropna=False)

spoken_languages
[{'iso_639_1': 'en', 'name': 'English'}]                                                                                                                                                                     22395
[]                                                                                                                                                                                                            3829
[{'iso_639_1': 'fr', 'name': 'Français'}]                                                                                                                                                                     1853
[{'iso_639_1': 'ja', 'name': '日本語'}]                                                                                                                                                                          1289
[{'iso_639_1': 'it', 'name': 'Italiano'}]                                                                                                  

Use both the "iso_639_1" and "name" properties to replace `spoken_languages`.

### Conclusion

Issues found:

1. **`genres` need to be unrolled. "name" property should be used for each `genre`.**
2. **`production_companies` need to be unrolled. "name" property should be used for each `production_company`.**
3. **`production_countries` need to be unrolled. "iso_3166_1" and "name" property should be used for each `production_country`.**
4. **`spoken_languages` need to be unrolled. "iso_639_1" and "name" property should be used for each `spoken_language`.**

In [28]:
df[~df['budget'].apply(lambda x: x.isnumeric())]

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
19730,- Written by Ørnås,0.065736,/ff9qCepilowshEtG2GYWwzt2bs4.jpg,"[{'name': 'Carousel Productions', 'id': 11176}...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",1997-08-20,0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29503,Rune Balot goes to a casino connected to the ...,1.931659,/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg,"[{'name': 'Aniplex', 'id': 2883}, {'name': 'Go...","[{'iso_3166_1': 'US', 'name': 'United States o...",2012-09-29,0,68.0,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Released,...,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35587,Avalanche Sharks tells the story of a bikini ...,2.185485,/zaSf5OG7V8X8gqFvly88zDdRm46.jpg,"[{'name': 'Odyssey Media', 'id': 17161}, {'nam...","[{'iso_3166_1': 'CA', 'name': 'Canada'}]",2014-01-01,0,82.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,...,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


## 5. Datatypes

Check the correctness of data types by looking at a typical correct row.

In [30]:
df.iloc[1]

adult                                                                False
belongs_to_collection                                                  NaN
budget                                                            65000000
genres                   [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...
homepage                                                               NaN
id                                                                    8844
imdb_id                                                          tt0113497
original_language                                                       en
original_title                                                     Jumanji
overview                 When siblings Judy and Peter discover an encha...
popularity                                                       17.015539
poster_path                               /vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg
production_companies     [{'name': 'TriStar Pictures', 'id': 559}, {'na...
production_countries     

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

### Conclusion

Incorrectness in the data types was caused by the incorrect three rows we found earlier. We will later use pandas to automate correction after removing these rows.

Issue found: **Incorrect data types**

# List of issues

1. NaN `title` and invalid `original_title`
2. Incorrect rows with ID 19730, 29503, and 35587
3. `genres` need to be unrolled. "name" property should be used for each `genre`.
5. `production_companies` need to be unrolled. "name" property should be used for each `production_company`.
6. `production_countries` need to be unrolled. "iso_3166_1" and "name" properties should be used for each `production_country`.
7. `spoken_languages` need to be unrolled. "iso_639_1" and "name" properties should be used for each `spoken_language`.
8. Drop unneeded columns
9. Incorrect data types

# Clean-ups

In [32]:
dfc = df.copy()

## 1. NaN `title` and invalid `original_title`

For all NaN `title`s:

- Use the `original_title` if `original_title` does not have JSON text
- Use `poster_path` otherwise, and also replace the `original_title` with `poster_path`

In [33]:
# Fill NaN titles using original_title if it does not contain '{', else use poster_path
# Create a mask for rows where title is NaN
nan_title_mask = dfc['title'].isna()

# Create a mask for original_title containing '{'
original_title_json_mask = dfc['original_title'].str.contains(r'{', regex=True)

# Apply conditions to fill NaN titles
dfc.loc[nan_title_mask & ~original_title_json_mask, 'title'] = dfc['original_title']
dfc.loc[nan_title_mask & original_title_json_mask, 'title'] = dfc['poster_path']

# Replace original_title with poster_path where original_title contains '{' and title was NaN
dfc.loc[nan_title_mask & original_title_json_mask, 'original_title'] = dfc['poster_path']

### Test

Let's run the same code as we had in the Assessment section. When correct, should return empty rows (meaning there's no empty `title` and no `original_title` with JSON content).

In [34]:
dfc[dfc['title'].isna()][['title', 'original_title', 'poster_path']]

,title,original_title,poster_path


In [35]:
dfc[dfc['original_title'].str.contains(r'{', regex=True)][['original_title', 'title']]

,original_title,title


## 2. Incorrect rows with ID 19730, 29503, and 35587

Drop rows with these IDs.

In [36]:
ids_to_remove = [19730, 29503, 35587]
dfc = dfc[~dfc.index.isin(ids_to_remove)].reset_index()

### Test

Check if there are still rows with invalid `belongs_to_collection` and `homepage`.

In [37]:
filled = dfc['belongs_to_collection'].fillna('')
dfc[(filled.fillna('') != '') & (~filled.str.contains(r'{', regex=True))] \
  [['belongs_to_collection', 'title', 'original_title', 'poster_path']]

,belongs_to_collection,title,original_title,poster_path


In [38]:
dfc['homepage'].value_counts()

homepage
http://www.georgecarlin.com                               12
http://www.wernerherzog.com/films-by.html                  7
http://breakblade.jp/                                      6
http://www.kungfupanda.com/                                4
http://www.missionimpossible.com/                          4
                                                          ..
http://happyfeettwo.warnerbros.com/index.html              1
http://rampartmovie.com/                                   1
http://singyoursongthemovie.com/                           1
http://extremelyloudandincrediblyclose.warnerbros.com/     1
http://www.promnightinmississippi.com/                     1
Name: count, Length: 7670, dtype: int64

Do we still have the value "False" in `production_companies`?

In [39]:
dfc[dfc['production_companies'] == 'False']

,index,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count


### 3. `genres` need to be exploded. "name" property should be used for each `genre`.

In [40]:
test_str = "[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"

# Extract the "name" property from JSON strings

def fix_json_string(json_str):
    json_str = json_str.replace("None", "null")
    
    # Replace single quotes around keys and values with double quotes, but preserve single quotes within values
    json_str = re.sub(r"'(?P<key>[^']+?)':", r'"\g<key>":', json_str)  # Keys
    json_str = re.sub(r":\s*'(.*?)'(,|})", r': "\g<1>"\2', json_str) # Values
    return json_str
    
def load_json(v):
    fixed_str = fix_json_string(v)
    return json.loads(fixed_str)
    
def str_separate(json_text, prop, separator='|'):
    obj = load_json(json_text)
    return separator.join([x['name'] for x in obj])

str_separate(test_str, 'name')

'Animation|Comedy|Family'

In [41]:
def split_var(_df, varname, new_varname, property='name'):
    """ Split a dataframe where a variable with multiple values is unrolled into multiple rows.
    
    Args:
    - df: The DataFrame object
    - varname: Current variable name
    - new_varname: New variable name that contains the singular item
    - property: The property name to get from each JSON object
    """
    temp_varname = "_{}_temp".format(varname)
    
    # Convert the JSON-like strings to lists of names
    s = _df[varname].apply(lambda x: str_separate(x, 'name', '|'))

    _df[temp_varname] = s.str.split('|')
    df1 = _df.explode(temp_varname).reset_index(drop=True)
    df1.rename(columns={temp_varname: new_varname}, inplace=True)
    _df.drop(columns=[temp_varname], inplace=True)
    return df1

start_time = time.time()
df_genres = split_var(dfc, 'genres', 'genre', property='name')[['id', 'genre']]
df_genres.rename(columns={'id': 'movie_id'}, inplace=True)

literal_eval_time = time.time() - start_time
print("time: {}".format(literal_eval_time))

time: 0.2787508964538574


## 4. `production_companies` need to be unrolled. "name" property should be used for each `production_company`.

In [42]:
def quick_load_json(v):
    """ Load JSON with literal eval """
    if type(v) != str and (v is None or np.isnan(v)):
        return []
    return ast.literal_eval(v)
    
def split_vars(_df, varname, properties=["name"], new_varnames=["_name"]):
    """ Split a dataframe where a variable with multiple values is unrolled into multiple rows.
    
    Args:
    - df: The DataFrame object
    - varname: Current variable name
    - properties: Properties to extract
    - new_varnames: New variable names that will contain each singular property value
    - separator: Separator in the variable
    """
    temp_varname = "_{}_temp".format(varname)
    _df[temp_varname] = _df[varname].apply(lambda x: quick_load_json(x))
    df1 = _df.explode(temp_varname)

    # Generalized function to extract specified properties from a dictionary
    def extract_properties(data_dict, properties):
        if pd.isna(data_dict):
            return pd.Series([None] * len(properties))
        try:
            # Extract the specified properties
            return pd.Series([data_dict.get(prop, None) for prop in properties])
        except (ValueError, SyntaxError, AttributeError):
            return pd.Series([None] * len(properties))
            
    df1[new_varnames] = df1[temp_varname].apply(extract_properties, properties=properties)
    df1.drop(columns=[temp_varname], inplace=True)
    _df.drop(columns=[temp_varname], inplace=True)
    return df1


start_time = time.time()

df_production_companies = split_vars(dfc, 'production_companies',
                properties=['name'],
                new_varnames=['name']
               )[['id', 'name']]
df_production_companies.rename(columns={'id': 'movie_id'}, inplace=True)

literal_eval_time = time.time() - start_time
print("time: {}".format(literal_eval_time))

time: 3.0145158767700195


### Test

In [43]:
df_production_companies

,movie_id,name
0,862,Pixar Animation Studios
1,8844,TriStar Pictures
1,8844,Teitler Film
1,8844,Interscope Communications
2,15602,Warner Bros.
...,...,...
45458,439050,None
45459,111109,Sine Olivia
45460,67758,American World Pictures
45461,227506,Yermoliev


## 5. `production_countries` need to be unrolled. "iso_3166_1" and "name" properties should be used for each `production_country`.

In [44]:
dfc.loc[0]['production_countries']

"[{'iso_3166_1': 'US', 'name': 'United States of America'}]"

In [45]:
test_str = "[{'iso_3166_1': 'US', 'name': 'United States of America'}, {'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso_3166_1': 'LU', 'name': 'Luxembourg'}]"
ast.literal_eval(test_str)

[{'iso_3166_1': 'US', 'name': 'United States of America'},
 {'iso_3166_1': 'CA', 'name': 'Canada'},
 {'iso_3166_1': 'LU', 'name': 'Luxembourg'}]

In [46]:
test_str = "[{'iso_3166_1': 'US', 'name': None}, {'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso_3166_1': 'LU', 'name': 'Luxembourg'}]"
ast.literal_eval(test_str)

[{'iso_3166_1': 'US', 'name': None},
 {'iso_3166_1': 'CA', 'name': 'Canada'},
 {'iso_3166_1': 'LU', 'name': 'Luxembourg'}]

In [47]:
# Split the var
start_time = time.time()
df_production_countries = split_vars(dfc, 'production_countries',
                properties=['iso_3166_1', 'name'],
                new_varnames=['iso_3166_1', 'name']
               )[['id', 'iso_3166_1', 'name']]
df_production_countries.rename(columns={'id': 'movie_id'}, inplace=True)

literal_eval_time = time.time() - start_time
print("time: {}".format(literal_eval_time))

time: 1.9772422313690186


### Test
Let's see some produced values.

In [48]:
df_production_countries.iloc[20:30]

,movie_id,iso_3166_1,name
14,1408,US,United States of America
15,524,FR,France
15,524,US,United States of America
16,4584,GB,United Kingdom
16,4584,US,United States of America
17,5,US,United States of America
18,9273,US,United States of America
19,11517,US,United States of America
20,8012,US,United States of America
21,1710,US,United States of America


## 6. `spoken_languages` need to be unrolled. "iso_639_1" and "name" properties should be used for each `spoken_language`.

In [49]:
start_time = time.time()
df_spoken_languages = split_vars(dfc, 'spoken_languages',
                properties=['iso_639_1', 'name'],
                new_varnames=['iso_639_1', 'name']
               )[['id', 'iso_639_1', 'name']]
df_spoken_languages.rename(columns={'id': 'movie_id'}, inplace=True)

literal_eval_time = time.time() - start_time
print("time: {}".format(literal_eval_time))

time: 2.0628626346588135


### Test

In [50]:
df_spoken_languages.iloc[20:30]

,movie_id,iso_639_1,name
13,10858,en,English
14,1408,en,English
14,1408,la,Latin
15,524,en,English
16,4584,en,English
17,5,en,English
18,9273,en,English
19,11517,en,English
20,8012,en,English
21,1710,en,English


## 7. Drop unneeded columns

In [51]:
dfc.drop(columns=['genres', 'production_companies', 'production_countries', 'spoken_languages'], inplace=True)

## 8. Incorrect data types

In [52]:
dfc = dfc.convert_dtypes()

### Test

In [53]:
dfc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45463 entries, 0 to 45462
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   index                  45463 non-null  Int64  
 1   adult                  45463 non-null  string 
 2   belongs_to_collection  4491 non-null   string 
 3   budget                 45463 non-null  string 
 4   homepage               7779 non-null   string 
 5   id                     45463 non-null  string 
 6   imdb_id                45446 non-null  string 
 7   original_language      45452 non-null  string 
 8   original_title         45463 non-null  string 
 9   overview               44509 non-null  string 
 10  popularity             45460 non-null  object 
 11  poster_path            45077 non-null  string 
 12  release_date           45376 non-null  string 
 13  revenue                45460 non-null  Int64  
 14  runtime                45203 non-null  Int64  
 15  st

In [54]:
dfc.iloc[100]

index                                                                  100
adult                                                                False
belongs_to_collection                                                 <NA>
budget                                                                   0
homepage                                                              <NA>
id                                                                   47475
imdb_id                                                          tt0117102
original_language                                                       en
original_title                                                   Mr. Wrong
overview                 A single and lonely woman finds the seemingly ...
popularity                                                        3.329264
poster_path                               /8oZW1loLEczN1BU60YobsJJFdSR.jpg
release_date                                                    1996-02-16
revenue                  

`budget` and `popularity` look incorrect, let's analyze by finding non-numeric values in them.

In [55]:
def is_non_numeric_string(x):
  if isinstance(x, str):
    try:
      float(x)
      return False
    except ValueError:
      return True
  return False

print("Non numeric strings in popularity:")
print(dfc[dfc['popularity'].apply(is_non_numeric_string)]['popularity'].count())
print("Non numeric strings in budget:")
print(dfc[dfc['budget'].apply(is_non_numeric_string)]['budget'].count())

Non numeric strings in popularity:
0
Non numeric strings in budget:
0


Seems like we can just convert `popularity` and `budget` to numeric data type.

## 9. (additional) Incorrect `popularity` and `budget` data types

In [56]:
dfc['popularity'] = pd.to_numeric(dfc['popularity'])
dfc['budget'] = pd.to_numeric(dfc['budget'])

### Test

In [57]:
dfc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45463 entries, 0 to 45462
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   index                  45463 non-null  Int64  
 1   adult                  45463 non-null  string 
 2   belongs_to_collection  4491 non-null   string 
 3   budget                 45463 non-null  Int64  
 4   homepage               7779 non-null   string 
 5   id                     45463 non-null  string 
 6   imdb_id                45446 non-null  string 
 7   original_language      45452 non-null  string 
 8   original_title         45463 non-null  string 
 9   overview               44509 non-null  string 
 10  popularity             45460 non-null  float64
 11  poster_path            45077 non-null  string 
 12  release_date           45376 non-null  string 
 13  revenue                45460 non-null  Int64  
 14  runtime                45203 non-null  Int64  
 15  st

Looks good! We don't need to correct the `release_date` data type since we are just going to save the DataFrame into a CSV file and analyze it in Tableau later.

# Save the final dataset

In [58]:
dfc.to_csv('movies_cleaned/movies_metadata.csv', index=False)
df_genres.to_csv('movies_cleaned/genres.csv', index=False)
df_production_companies.to_csv('movies_cleaned/production_companies.csv', index=False)
df_spoken_languages.to_csv('movies_cleaned/spoken_languages.csv', index=False)

In [64]:
# Save to Zip to make them easier to download
import zipfile
import os

# Function to add files to a zip file
def create_zip_file(zip_filename, files_to_zip):
    with zipfile.ZipFile(zip_filename, 'w') as zipf:
        for file in files_to_zip:
            if os.path.isfile(file):
                zipf.write(file, os.path.basename(file))
            else:
                print(f"File {file} does not exist and will be skipped.")

# List of files to be zipped
files_to_zip = ['movies_cleaned/movies_metadata.csv',
                'movies_cleaned/genres.csv',
                'movies_cleaned/production_companies.csv',
                'movies_cleaned/spoken_languages.csv']

# Name of the output zip file
zip_filename = 'movies_cleaned/movies_metadata.zip'

# Create the zip file
create_zip_file(zip_filename, files_to_zip)

print(f"{zip_filename} created successfully.")

movies_cleaned/movies_metadata.zip created successfully.


# Analysis

In [59]:
df[df['title'].str.contains("Absolutions", na=False)]

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
24420,False,NaN,0,[],NaN,292445,tt2316325,en,(Entlastungen) Pipilottis Fehler,Precisely edited to the start-stop rhythm of a...,...,1988-01-13,0.0,11.0,"[{'iso_639_1': 'de', 'name': 'Deutsch'}]",Released,NaN,(Absolutions) Pipilotti's Mistakes,False,0.0,0.0


# Appendix 1. Flattening a Table with Multiple Columns Containing Multiple Values

The example below demonstrates how to flatten a table with multiple columns that contain multiple values. The goal is to avoid creating a Cartesian product of the columns. We provide two approaches: a straightforward looping approach and a more efficient vectorized approach.

**However, flattening columns still present another issue: We can no longer accurately calculate aggregated numeric columns. Therefore, we decided to create relational tables instead.**

## Looping Approach (Slower)

The first code block uses the straightforward looping approach, which, while intuitive, can be slow for large datasets.

In [60]:
# Example DataFrame with multiple fields having multiple values
_df = pd.DataFrame({
    'ID': [1, 2, 3],
    'Name': ['Alice', 'Bob', 'Charlie'],
    'Hobbies': [['Reading', 'Swimming'], ['Cycling'], ['Running', 'Cycling', 'Reading']],
    'Skills': [['Python', 'SQL'], ['Excel'], ['Python', 'Tableau']]
})

# Flatten the DataFrame
flattened_data = []

for _, row in _df.iterrows():
    max_len = max(len(row['Hobbies']), len(row['Skills']))
    hobbies = row['Hobbies'] + [None] * (max_len - len(row['Hobbies']))
    skills = row['Skills'] + [None] * (max_len - len(row['Skills']))
    
    for hobby, skill in zip(hobbies, skills):
        flattened_data.append([row['ID'], row['Name'], hobby, skill])

# Create a new flattened DataFrame
flattened_df = pd.DataFrame(flattened_data, columns=['ID', 'Name', 'Hobby', 'Skill'])

flattened_df

,ID,Name,Hobby,Skill
0,1,Alice,Reading,Python
1,1,Alice,Swimming,SQL
2,2,Bob,Cycling,Excel
3,3,Charlie,Running,Python
4,3,Charlie,Cycling,Tableau
5,3,Charlie,Reading,None


## Vectorized Approach (Faster)
The second code block demonstrates a more efficient, vectorized approach that avoids the pitfalls of the Cartesian product while maintaining performance.

In [61]:
# Example DataFrame with multiple fields having multiple values
_df = pd.DataFrame({
    'ID': [1, 2, 3],
    'Name': ['Alice', 'Bob', 'Charlie'],
    'Hobbies': [['Reading', 'Swimming'], ['Cycling'], ['Running', 'Cycling', 'Reading']],
    'Skills': [['Python', 'SQL'], ['Excel'], ['Python', 'Tableau']]
})

# Explode each column individually
hobbies_exploded = _df.explode('Hobbies').reset_index(drop=True)
skills_exploded = _df.explode('Skills').reset_index(drop=True)

# Ensure the length of both exploded dataframes match
max_len = max(len(hobbies_exploded), len(skills_exploded))
hobbies_exploded = hobbies_exploded.reindex(range(max_len)).reset_index(drop=True)
skills_exploded = skills_exploded.reindex(range(max_len)).reset_index(drop=True)

# Combine the exploded columns
flattened_df = hobbies_exploded[['ID', 'Name', 'Hobbies']].combine_first(skills_exploded[['ID', 'Name', 'Skills']])

# Rename columns for clarity
flattened_df = flattened_df.rename(columns={'Hobbies': 'Hobby', 'Skills': 'Skill'})

flattened_df

,Hobby,ID,Name,Skill
0,Reading,1.0,Alice,Python
1,Swimming,1.0,Alice,SQL
2,Cycling,2.0,Bob,Excel
3,Running,3.0,Charlie,Python
4,Cycling,3.0,Charlie,Tableau
5,Reading,3.0,Charlie,NaN


### Test

Take a quick look at the produced dataset.

In [62]:
df_genres.head(10)

,movie_id,genre
0,862,Animation
1,862,Comedy
2,862,Family
3,8844,Adventure
4,8844,Fantasy
5,8844,Family
6,15602,Romance
7,15602,Comedy
8,31357,Comedy
9,31357,Drama
